In [40]:
import nltk
import torch
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
nltk.download('stopwords')
stop_words = set(stopwords.words('russian'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\asdsf\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [34]:
questions = [
    "Привет, как дела?",
    "Какие у тебя хобби?",
    "Что ты думаешь о искусственном интеллекте?",
    "Какой твой любимый цвет?"
]

answers = [
    "Привет! У меня все отлично, спасибо!",
    "Я люблю общаться с людьми и узнавать новые вещи.",
    "Искусственный интеллект очень увлекательная тема, не так ли?",
    "Мой любимый цвет - синий."
]

In [35]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(questions)

In [36]:
def get_response(user_question):
    question_tfidf = vectorizer.transform([user_question])
    similarities = cosine_similarity(question_tfidf, X)
    index = np.argmax(similarities)
    return answers[index]



In [37]:
user_question = "Как твои дела?"
response = get_response(user_question)
print(response)

Привет! У меня все отлично, спасибо!


In [38]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

C:\Users\asdsf\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [41]:
model_name = "sberbank-ai/rugpt3small_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

model.eval()

tokenizer_config.json: 100%|██████████| 1.25k/1.25k [00:00<00:00, 1.30MB/s]
vocab.json: 100%|██████████| 1.71M/1.71M [00:02<00:00, 682kB/s]
merges.txt: 100%|██████████| 1.27M/1.27M [00:00<00:00, 2.88MB/s]
config.json: 100%|██████████| 720/720 [00:00<?, ?B/s] 
pytorch_model.bin: 100%|██████████| 551M/551M [01:50<00:00, 4.99MB/s] 


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50264, 768)
    (wpe): Embedding(2048, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50264, bias=False)
)

In [45]:
def generate_response(prompt, max_length=50):
    input_ids = tokenizer.encode(prompt, return_tensors="pt")

    # Generate response
    with torch.no_grad():
        output = model.generate(input_ids, max_length=max_length, num_return_sequences=1, pad_token_id=50256)

    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response

In [46]:
print("Славик: Приветик, как делишки?")
while True:
    user_input = input("Я: ")
    if user_input.lower() == "ухожу":
        print("Славик: Покаааааааа!")
        break

    response = generate_response(user_input)
    print("Славик:", response)

Chatbot: Hi there! How can I help you?
Chatbot: Как дела?

— Да, — ответил я. — Я в порядке.

— Я тоже.

— Я тоже.

— Я тоже.

— Я тоже.


Chatbot: как оно?

— Я не знаю, — ответил он. — Я не знаю, что это такое.

— Я знаю, — сказала она. — Я знаю, что это такое.

—
Chatbot: У тебя всё хорошо?

— Всё хорошо, — ответил я. — Я не хочу, чтобы ты уходила.

— Я не уйду, — повторила она. — Я не хочу, чтобы ты уходил.


IndexError: index -1 is out of bounds for dimension 1 with size 0

Сейчас будет скам (симбиоз tf-idf и трансформера))

In [47]:
from transformers import pipeline 

In [65]:
QandA = pd.read_csv('dialog.csv')
QandA.head()

,question,answer
0,"hi, how are you doing?",i'm fine. how about yourself?
1,i'm fine. how about yourself?,i'm pretty good. thanks for asking.
2,i'm pretty good. thanks for asking.,no problem. so how have you been?
3,no problem. so how have you been?,i've been great. what about you?
4,i've been great. what about you?,i've been good. i'm in school right now.


In [69]:
questions = QandA['question'].tolist()
answers = QandA['answer'].tolist()

In [70]:
X = vectorizer.fit_transform(questions)
X_transformers = vectorizer.transform(answers)

In [71]:
transformer_pipeline = pipeline("text-generation", model="distilgpt2")
for question in questions:
    inputs = question + " "
    response = transformer_pipeline(inputs, max_length=50, num_return_sequences=1)
    print("User:", question)
    print("Chatbot:", response[0]['generated_text'])

config.json: 100%|██████████| 762/762 [00:00<00:00, 761kB/s]
model.safetensors: 100%|██████████| 353M/353M [01:03<00:00, 5.51MB/s] 
generation_config.json: 100%|██████████| 124/124 [00:00<00:00, 124kB/s]
vocab.json: 100%|██████████| 1.04M/1.04M [00:00<00:00, 1.74MB/s]
merges.txt: 100%|██████████| 456k/456k [00:00<00:00, 1.28MB/s]
tokenizer.json: 100%|██████████| 1.36M/1.36M [00:00<00:00, 5.28MB/s]
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


User: hi, how are you doing?
Chatbot: hi, how are you doing? ????

-
-
.
-.
-
-
-
-
1 -.
-
-
-
-
-
-
-
-
-
-


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


User: i'm fine. how about yourself?
Chatbot: i'm fine. how about yourself? _____-Mii
* * * *
"You will live this great life." ―Mii
"You will be here tomorrow." ―Mii
"Here." ―Mii
"


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


User: hello, may i speak to alice please?
Chatbot: hello, may i speak to alice please? ***************************


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


User: this is she. how's it going?
Chatbot: this is she. how's it going? ㅗㅗㅗㅗㅗ

So, now you're ready for this…


KeyboardInterrupt: 